## Import Libraries

In [ ]:
import random
import string
import numpy as np
from PIL import Image

## Define Helper Functions for encoding the message in image

In [ ]:
def string_to_binary(message):
    return ''.join(format(ord(c), '08b') for c in message)

def encode_lsb_row_wise(image, binary_message, bits=1):
    encoded_image = image.copy()
    height, width, channels = encoded_image.shape
    bin_len = len(binary_message)
    
    total_pixels = height * width * channels
    random_start = random.randint(0, max(0, total_pixels - (bin_len // bits) - 10))
    flat_image = encoded_image.reshape(-1)
    
    max_bits = (len(flat_image) - random_start) * bits
    if len(binary_message) > max_bits:
        binary_message = binary_message[:max_bits]
    
    index = random_start
    binary_index = 0
    
    while binary_index < len(binary_message) and index < len(flat_image):
        bit_end = min(binary_index + bits, len(binary_message))
        bits_to_encode = binary_message[binary_index:bit_end]
        
        bits_to_encode = bits_to_encode.ljust(bits, '0')
        
        binary_value = int(bits_to_encode, 2)
        
        if bits == 8:
            flat_image[index] = binary_value
        else:
            mask = 256 - (2**bits)
            pixel_value = flat_image[index]
            pixel_value = (pixel_value & mask) | binary_value
            flat_image[index] = pixel_value
        
        index += 1
        binary_index += bits
    
    encoded_image = flat_image.reshape(height, width, channels)
    return encoded_image, random_start

## Encodes a message into an image and saves the encoded image to the specified output path.

In [ ]:
def encode_image(image_path, message, output_path, bits=1):
    image = Image.open(image_path)
    image = np.array(image)
    
    binary_message = string_to_binary(message) 
    
    encoded_image, random_start = encode_lsb_row_wise(image, binary_message, bits)
    encoded_pil_image = Image.fromarray(encoded_image)
    encoded_pil_image.save(output_path)
    
    print(f"Encoded image saved at {output_path} with random start index {random_start}")

## We are resizing the image to decode the images fastly (for demostrating purpose)

In [1]:
def resize_image(image_path, output_path, size=(256, 256)):
    image = Image.open(image_path)
    resized_image = image.resize(size)
    resized_image.save(output_path)
    print(f"Resized image saved at {output_path} with size {size}")

In [45]:
resize_image("./dataset_4/cover/01158.png", "./test/4.png", size=(32, 32))

Resized image saved at ./test/4.png with size (32, 32)


In [23]:
encode_image('./test/4.png', 'Rahul is at location x', './test/encoded_image4.png', bits=4)

Encoded image saved at ./test/encoded_image4.png with random start index 241


## Brute force decoding

In [1]:
import numpy as np
from PIL import Image
import string

def extract_lsb_bits(arr, lsb_count):
    mask = (1 << lsb_count) - 1
    bits = np.unpackbits(arr & mask, axis=None)
    bits = bits.reshape(-1, 8)[:, -lsb_count:].flatten()
    return bits

def bits_to_bytes(bits):
    n = len(bits) // 8
    bits = bits[:n*8]
    return np.packbits(bits).tobytes()

def is_printable(s, threshold=0.9):
    printable = set(string.printable.encode())
    return sum(c in printable for c in s) / len(s) > threshold

def decode_lsb(image_path, min_msg_len=8, max_msg_len=64):
    img = Image.open(image_path)
    arr = np.array(img).flatten()
    results = []

    for lsb_count in range(1, 9):
        bits = extract_lsb_bits(arr, lsb_count)
        for msg_len in range(min_msg_len, max_msg_len+1):
            total_bits = msg_len * 8
            for start in range(0, len(bits) - total_bits + 1):
                msg_bits = bits[start:start+total_bits]
                msg_bytes = bits_to_bytes(msg_bits)
                if is_printable(msg_bytes):
                    try:
                        msg = msg_bytes.decode(errors='ignore')
                        results.append((lsb_count, start, msg))
                    except Exception:
                        continue
    return results

## Decode LSB Example

In [2]:
results = decode_lsb('./test/encoded_image5.png')

In [ ]:
results